# <font color = 'indianred'> **1. Setting up the Environment** </font>



In [ ]:
# If in Colab, then import the drive module from google.colab
if 'google.colab' in str(get_ipython()):
  !pip install numpy -U -qq
  !pip install transformers evaluate wandb datasets accelerate trl peft bitsandbytes -U -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 144.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.

 <Font size = 5 color = 'indianred'>**Restart the session before moving onto next cell**
> Runtime- Restart Session

In [ ]:
!pip install -Uqq torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


<font color = 'indianred'> *Load Libraries* </font>

In [ ]:
# standard data science librraies for data handling and v isualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
import gc
import time

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, precision_score, recall_score, f1_score
import joblib

import torch
import torch.nn as nn
import ast


# New libraries introduced in this notebook
import evaluate
from datasets import load_dataset, DatasetDict, Dataset, ClassLabel
from transformers import (
    TrainingArguments,
    Trainer,
    set_seed,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoConfig,
    pipeline,
    BitsAndBytesConfig,
)

from trl import SFTTrainer
from peft import (
    TaskType,
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    AutoPeftModelForSequenceClassification,
    PeftConfig
)
import wandb
from google.colab import userdata
from huggingface_hub import login, HfApi, create_repo

In [ ]:
wandb_api_key = userdata.get('WANDB_API_KEY')
hf_token = userdata.get('HF_Token')


In [ ]:
if hf_token:
    # Log in to Hugging Face
    login(token=hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Hugging Face token not found in notebook secrets.")


Successfully logged in to Hugging Face!


In [ ]:
if wandb_api_key:
  wandb.login(key=wandb_api_key)
  print("Successfully logged in to WANDB!")
else:
    print("WANDB key not found in notebook secrets.")


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ronchaudhuri29 (ronchaudhuri29-the-university-of-texas-at-dallas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Successfully logged in to WANDB!


# <font color = 'indianred'> **2. Load Data set**
    


In [ ]:
# Determine the storage location based on the execution environment
# If running on Google Colab, use Google Drive as storage
# CHANGE FOLDERS TO WHERE YOU WANT TO SAVE DATA AND MODELS

if 'google.colab' in str(get_ipython()):
    from google.colab import drive  # Import Google Drive mounting utility
    drive.mount('/content/drive')  # Mount Google Drive

    # Set base folder path for storing data on Google Drive
    base_folder= Path('/content/drive/MyDrive/data')



Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_folder = base_folder/'datasets'
import pandas as pd

train_path = data_folder / 'train.csv'
test_path  = data_folder / 'test.csv'
sub_path   = data_folder/'sample_submission.csv'


train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)
sample_sub = pd.read_csv(sub_path)

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
train_df.head()




Train shape: (2029, 9)
Test shape: (10, 8)


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1


In [ ]:
import pandas as pd

# --- Helper: safe get + stringify + prefix label ---
def _fmt(prefix, val):
    if pd.isna(val) or (isinstance(val, str) and not val.strip()):
        return ""
    s = str(val).replace("\n", " ").strip()
    return f"{prefix}{s} | "

# Columns to combine (use only if they exist)
CTX_COLUMNS = [
    ("Comment: ", "body"),
    ("Subreddit: ", "subreddit"),
    ("Rule: ", "rule"),
    ("Positive ex1: ", "positive_example_1"),
    ("Positive ex2: ", "positive_example_2"),
    ("Negative ex1: ", "negative_example_1"),
    ("Negative ex2: ", "negative_example_2"),
]


def combine_row(row):
    parts = []
    for prefix, col in CTX_COLUMNS:
        if col in row:
            parts.append(_fmt(prefix, row[col]))
    return "".join(parts) or "Comment: "

# --- Build combined text ---
train_df["text"] = train_df.apply(combine_row, axis=1)
test_df["text"]  = test_df.apply(combine_row, axis=1)

# --- Map target & ids ---
train_df = train_df.rename(columns={"rule_violation": "label"})

# --- Ensure binary labels ---
train_df["label"] = pd.to_numeric(train_df["label"], errors="raise").astype(int)
unique_labels = set(train_df["label"].unique())
assert unique_labels <= {0, 1}, f"Expected binary labels 0/1, got {unique_labels}"

print(" Sample combined text:")
print(train_df["text"].iloc[0][:500])


 Sample combined text:
Comment: Banks don't want you to know this! Click here to know more! | Subreddit: Futurology | Rule: No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. | Positive ex1: If you could tell your younger self something different about sex, what would that be?  i AM IN A CONTEST TO WIN FUNDING FOR MY SEX POSITIVE FILM: VOTE HERE:  http://sheknows.offerpop.com/campaign/813112/entry/v144417 | Positive ex2: hunt for lady for jack off in neighbourhood h


# <font color = 'indianred'> **3. Accessing and Manuplating Splits**</font>



<font color = 'indianred'>*Create futher subdivions of the splits*</font>

In [ ]:
# ---- Part A: train/val split and HF datasets ----
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Split TRAIN → train/val (stratified by your binary label)
train_split, val_split = train_test_split(
    train_df[["row_id", "text", "label"]],
    test_size=0.2,
    random_state=42,
    stratify=train_df["label"]
)

# Convert to HF datasets (keep only text+label for training)
train_ds = Dataset.from_pandas(train_split[["text","label"]].reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_split[["text","label"]].reset_index(drop=True))

# Keep row_id for submission alignment on test
test_ds  = Dataset.from_pandas(test_df[["row_id","text"]].reset_index(drop=True))

print(train_ds, val_ds, test_ds)


Dataset({
    features: ['text', 'label'],
    num_rows: 1623
}) Dataset({
    features: ['text', 'label'],
    num_rows: 406
}) Dataset({
    features: ['row_id', 'text'],
    num_rows: 10
})


<font color = 'indianred'>*small subset for initial experimenttaion*</font>

In [ ]:
#train_split = train_split.shuffle(seed = 42).select(range(2000))
#val_split = val_split.shuffle(seed = 42).select(range(2000))
#test_split = test_split.shuffle(seed = 42).select(range(2000))

In [ ]:
train_val_subset= DatasetDict(
    {"train": train_split, "valid": val_split})

In [ ]:
train_val_subset

DatasetDict({
    train:       row_id                                               text  label
    1109    1109  Comment: Well first you have to figure out if ...      1
    490      490  Comment: cheap cigarettes online Buy Discount ...      1
    159      159  Comment: It's illegal, you can sue for pain an...      1
    333      333  Comment: You should be fine. There have been c...      1
    292      292  Comment: [Also Watch This Video - Olympics](ht...      0
    ...      ...                                                ...    ...
    467      467  Comment: #Rapper  🚨Straight Outta Cross Keys S...      0
    1759    1759  Comment: I don't know the specifics of the que...      1
    536      536  Comment: Selling Tyrande codes for 3€ to paypa...      1
    677      677  Comment: If anyone is interested in donating 5...      0
    1714    1714  Comment: look that pussy will  and be in on co...      1
    
    [1623 rows x 3 columns]
    valid:       row_id                       

In [ ]:
train_val_subset['train']['label'][0]

np.int64(0)

# <font color = 'indianred'>**4. Load pre-trained Tokenizer**</font>



In [ ]:
def free_gpu_memory():
    """
    Frees up GPU memory after CUDA out-of-memory error in Colab.

    This function performs the following steps:
    1. Deletes all PyTorch objects to clear references.
    2. Calls garbage collection to remove unreferenced objects from memory.
    3. Uses torch.cuda.empty_cache() to release cached GPU memory.
    4. Waits for a moment to ensure memory is fully released.
    """
    try:
        # Delete all torch tensors to free up memory
        for obj in list(locals().values()):
            if torch.is_tensor(obj):
                del obj

        # Collect garbage to release any remaining unused memory
        gc.collect()

        # Empty the CUDA cache to release GPU memory
        torch.cuda.empty_cache()

        # Adding a small delay to allow memory to be fully released
        time.sleep(2)

        print("GPU memory has been freed.")
    except Exception as e:
        print(f"Error while freeing GPU memory: {e}")


In [ ]:
free_gpu_memory()

GPU memory has been freed.


In [ ]:
checkpoint = "google/gemma-2-2b"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
lengths = [len(tokenizer(text)["input_ids"]) for text in train_df["text"][:1000]]
print("Average:", sum(lengths)/len(lengths))
print("Max:", max(lengths))

Average: 279.842
Max: 616


#<font color = 'indianred'> **5. Create function for Tokenizer**



In [ ]:
def tokenize_fn(batch):
    return tokenizer(text = batch["text"], truncation=True, max_length=512)

In [ ]:
from datasets import Dataset, DatasetDict

# 1) Make sure splits are HF Datasets (not pandas DataFrames)
def ensure_hf(ds, keep_cols):
    # if it's already a Dataset, just select needed columns
    from datasets import Dataset as HFDataset
    if isinstance(ds, HFDataset):
        return ds.select_columns(keep_cols)
    else:
        # assume pandas DataFrame
        return Dataset.from_pandas(ds[keep_cols].reset_index(drop=True))

train_hf = ensure_hf(train_split, ["text", "label"])
val_hf   = ensure_hf(val_split,   ["text", "label"])

# If you have test_split with row_id + text:
try:
    test_hf  = ensure_hf(test_ds,  ["row_id", "text"])
except NameError:
    test_hf = None

# 2) Tokenize each split separately (remove raw text; on test also drop row_id before model)
def tokenize_fn(batch):
    return tokenizer(text=batch["text"], truncation=True, max_length=512)

tokenized_train = train_hf.map(tokenize_fn, batched=True, remove_columns=["text"])
tokenized_val   = val_hf.map(tokenize_fn,   batched=True, remove_columns=["text"])

if test_hf is not None:
    test_ids = test_hf["row_id"]                  # keep ids for submission
    tokenized_test = test_hf.map(tokenize_fn, batched=True, remove_columns=["text","row_id"])

# 3) Rebuild a proper DatasetDict and set PyTorch format
tokenized_dataset = DatasetDict({
    "train": tokenized_train,
    "valid": tokenized_val
})
tokenized_dataset = tokenized_dataset.with_format("torch")

# (optional) keep tokenized_test separate for prediction
if test_hf is not None:
    tokenized_test = tokenized_test.with_format("torch")

print(tokenized_dataset)


Map:   0%|          | 0/1623 [00:00<?, ? examples/s]

Map:   0%|          | 0/406 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1623
    })
    valid: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 406
    })
})


<font color = 'indianred'> *Use map function to apply tokenization to all splits*

In [ ]:
tokenized_dataset = (tokenized_dataset
          .map(lambda x : {"float_label": x["label"].to(torch.float)}, remove_columns=["label"])
          .rename_column("float_label", "label"))

Map:   0%|          | 0/1623 [00:00<?, ? examples/s]

Map:   0%|          | 0/406 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset['train']['label'][0:3]

tensor([1., 1., 1.])

#  <font color = 'indianred'> **6. Model Training**

##  <font color = 'indianred'> **6.1 Download pre-trained model**

In [ ]:
def get_appropriate_dtype():
    if torch.cuda.is_available() and torch.cuda.get_device_capability(0) >= (8, 0):
        return torch.bfloat16
    return torch.float16

In [ ]:
torch_data_type = get_appropriate_dtype()
torch_data_type

torch.float16

In [ ]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_use_double_quant=True,
  bnb_4bit_compute_dtype=torch_data_type,
  bnb_4bit_quant_storage=torch_data_type,
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           num_labels = 1,
                                                           problem_type="single_label_classification",
                                                           quantization_config=bnb_config,
                                                           dtype=torch_data_type,
                                                           trust_remote_code=True,)


config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at google/gemma-2-2b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

Gemma2ForSequenceClassification(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear4bit(in_features=9216, out_features=2304, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
      

##  <font color = 'indianred'> **6.3 PEFT Setup**

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

# Typical Gemma targets
lora_targets = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

peft_cfg = LoraConfig(
    r=64,
    lora_alpha=128,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=lora_targets,
)

model = get_peft_model(model, peft_cfg)
model.print_trainable_parameters()


trainable params: 83,069,184 || all params: 2,697,413,376 || trainable%: 3.0796


In [ ]:
import transformers
from transformers import TrainingArguments
print("transformers version:", transformers.__version__)
print("TrainingArguments from:", TrainingArguments.__module__)

transformers version: 4.57.1
TrainingArguments from: transformers.training_args


In [ ]:
from datasets import Value
tokenized_dataset = tokenized_dataset.cast_column("label", Value("float32"))


Casting the dataset:   0%|          | 0/1623 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/406 [00:00<?, ? examples/s]

In [ ]:
model.config.use_cache = False


In [ ]:
import torch
from transformers import Trainer

class BCETrainer(Trainer):
    # accept the newer kwarg
    def compute_loss(
        self,
        model,
        inputs,
        return_outputs: bool = False,
        num_items_in_batch: int | None = None,
        **kwargs,
    ):
        labels = inputs.pop("labels").float()
        outputs = model(**inputs)
        logits = outputs.logits.squeeze(-1)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [ ]:
import numpy as np, evaluate
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers.trainer_utils import IntervalStrategy


# --- metrics (sigmoid -> 0/1) ---
acc  = evaluate.load("accuracy")
prec = evaluate.load("precision")
rec  = evaluate.load("recall")
f1   = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits.reshape(-1)))
    preds = (probs >= 0.5).astype(int)
    labels = labels.astype(int)
    return {
        "accuracy":  acc.compute(predictions=preds, references=labels)["accuracy"],
        "precision": prec.compute(predictions=preds, references=labels, average="binary")["precision"],
        "recall":    rec.compute(predictions=preds, references=labels, average="binary")["recall"],
        "f1":        f1.compute(predictions=preds, references=labels, average="binary")["f1"],
    }

data_collator = DataCollatorWithPadding(tokenizer)
training_args = TrainingArguments(
    output_dir="gemma_lora_binary_out",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    eval_strategy=IntervalStrategy.EPOCH,
    save_strategy=IntervalStrategy.EPOCH,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,
    gradient_checkpointing=True,
    report_to=["wandb"],
    run_name="partA_gemma_lora_bce_cosine"
)



trainer = BCETrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    processing_class=tokenizer,          # ← replaces tokenizer=tokenizer
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.546825,0.711823,0.802721,0.572816,0.668555
2,No log,0.449340,0.793103,0.752066,0.883495,0.812500
3,1.149900,0.478972,0.822660,0.810185,0.849515,0.829384
4,1.149900,0.571958,0.817734,0.805556,0.844660,0.824645


{'eval_loss': 0.47897249460220337,
 'eval_accuracy': 0.8226600985221675,
 'eval_precision': 0.8101851851851852,
 'eval_recall': 0.8495145631067961,
 'eval_f1': 0.8293838862559242,
 'eval_runtime': 63.2136,
 'eval_samples_per_second': 6.423,
 'eval_steps_per_second': 0.807,
 'epoch': 4.0}

In [ ]:
from scipy.special import expit

def to_probs(pred_array):
    """
    Convert logits to probabilities exactly once.
    If inputs already look like probabilities [0,1], return them unchanged.
    """
    x = np.asarray(pred_array).squeeze()
    if x.ndim == 0:
        x = np.array([x])
    if np.all((0.0 <= x) & (x <= 1.0)):   # already probabilities
        return x
    return expit(x)                      # logits -> probs (single sigmoid)


In [ ]:
# ---- VALIDATION EVAL (NO DOUBLE SIGMOID) ----
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score

# 1) Get logits & labels from the trained trainer
pred_val = trainer.predict(tokenized_dataset["valid"])
logits   = pred_val.predictions                 # shape: (N, 1) or (N,)
labels   = pred_val.label_ids.astype(int)

# 2) Convert logits -> probabilities ONCE (safe helper you added)
probs = to_probs(logits)                        # returns shape (N,)

# 3) Quick scan to choose best threshold by F1 (optional: tighten/expand range)
ts = np.linspace(0.20, 0.80, 61)
best_t, best_f1 = 0.5, -1.0
for t in ts:
    preds_t = (probs >= t).astype(int)
    f1_t = f1_score(labels, preds_t)
    if f1_t > best_f1:
        best_f1, best_t = f1_t, t

print(f"Best F1 on validation = {best_f1:.6f} at threshold = {best_t:.3f}")

# 4) Reports at default 0.5 and at best_t
for name, t in [("0.5", 0.5), (f"{best_t:.3f}", best_t)]:
    preds_t = (probs >= t).astype(int)
    print("\n" + "="*80)
    print(f"Classification report @ threshold={name}")
    print(classification_report(labels, preds_t, digits=4))
    print("Confusion matrix:")
    print(confusion_matrix(labels, preds_t))
    print(f"ROC-AUC (threshold-free): {roc_auc_score(labels, probs):.6f}")


Best F1 on validation = 0.834467 at threshold = 0.330

Classification report @ threshold=0.5
              precision    recall  f1-score   support

           0     0.8368    0.7950    0.8154       200
           1     0.8102    0.8495    0.8294       206

    accuracy                         0.8227       406
   macro avg     0.8235    0.8223    0.8224       406
weighted avg     0.8233    0.8227    0.8225       406

Confusion matrix:
[[159  41]
 [ 31 175]]
ROC-AUC (threshold-free): 0.890316

Classification report @ threshold=0.330
              precision    recall  f1-score   support

           0     0.8713    0.7450    0.8032       200
           1     0.7830    0.8932    0.8345       206

    accuracy                         0.8202       406
   macro avg     0.8272    0.8191    0.8189       406
weighted avg     0.8265    0.8202    0.8191       406

Confusion matrix:
[[149  51]
 [ 22 184]]
ROC-AUC (threshold-free): 0.890316


In [ ]:
import os, json
ART_DIR = "partA_cls_artifacts"
os.makedirs(ART_DIR, exist_ok=True)

model.save_pretrained(os.path.join(ART_DIR, "lora_adapter"))
tokenizer.save_pretrained(os.path.join(ART_DIR, "tokenizer"))

with open(os.path.join(ART_DIR, "inference_config.json"), "w") as f:
    json.dump({"best_threshold": float(0.33)}, f, indent=2)

print("Saved model + tokenizer + threshold →", ART_DIR)


Saved model + tokenizer + threshold → partA_cls_artifacts


In [ ]:


pred_test  = trainer.predict(tokenized_test)
test_probs = to_probs(pred_test.predictions)
test_preds = (test_probs >= best_t).astype(int)

sub = sample_sub.copy()

if "rule_violation" in sub.columns:
    sub["rule_violation"] = test_preds
elif "label" in sub.columns:
    sub["label"] = test_preds
else:

    sub["label"] = test_preds

SUB_PATH = "submission_partA_gemma2_lora.csv"
sub.to_csv(SUB_PATH, index=False)
print(f"Saved submission to: {SUB_PATH}")


Saved submission to: submission_partA_gemma2_lora.csv
